In [2]:
!ls ../data

unet_brainMRI_seg.hdf5


In [3]:
import h5py

In [8]:
filename = './../data/unet_brainMRI_seg.hdf5'

In [10]:
with h5py.File(filename, "r") as f:
    # Print all root level object names (aka keys) 
    # these can be group or dataset names 
    print("Keys: %s" % f.keys())
    # get first object name/key; may or may NOT be a group
    a_group_key = list(f.keys())[0]

    # get the object type for a_group_key: usually group or dataset
    print(type(f[a_group_key])) 

    # If a_group_key is a group name, 
    # this gets the object names in the group and returns as a list
    data = list(f[a_group_key])

    # If a_group_key is a dataset name, 
    # this gets the dataset values and returns as a list
    data = list(f[a_group_key])
    # preferred methods to get dataset values:
    ds_obj = f[a_group_key]      # returns as a h5py dataset object

Keys: <KeysViewHDF5 ['model_weights', 'optimizer_weights']>
<class 'h5py._hl.group.Group'>
